In [129]:
import itertools
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections
import re

# Challenge: https://adventofcode.com/2020/day/7

# Import csv
data_file_loc = 'C:/Users/jlahrman/OneDrive - LMI/Documents/Advent_of_Code/Advent_of_Code/Week05_Day07/'

column = ['luggage_rules']
day7data = pd.read_csv(data_file_loc + 'day_07_input.csv', 
                       header = None, 
                       names = column, 
                       skip_blank_lines=False
                      )

bag_parents = []
bag_children = []
luggage_rule_string = 'contain'
characters_to_remove = [' bags', ' bag', '.','1','2','3','4','5','6','7','8','9','  ']

for x in range(len(day7data)):
    # First, find the position of the separator between key and value
    position = day7data.iloc[x]['luggage_rules'].index(luggage_rule_string)
    # Now take the left-most number of characters based on that value
    bag_parent = day7data.iloc[x]['luggage_rules'][:position].strip()
    # The value is made up of the right characters, probably an easier way to do this but we're rolling with it
    bag_child = day7data.iloc[x]['luggage_rules'][-(len(day7data.iloc[x]['luggage_rules'])-position-len(luggage_rule_string)):].strip()
    # From both columns, remove a list of strings that we'll no longer need - ' bag', ' bags','.', all numbers
    # Has to be an easier way!
    for char in characters_to_remove:
        bag_parent = bag_parent.replace(char, '').strip()
        bag_child = bag_child.replace(char, '').strip()
#    rule_key.translate({ord(z): '' for z in characters_to_remove})
#    rule_key = re.sub("|".join(characters_to_remove), "", rule_key)
#    rule_value = re.sub("|".join(characters_to_remove), "", rule_value)
    # turn the values into a list
    bag_child = bag_child.split(',')
    bag_parents.append(bag_parent)
    bag_children.append(bag_child)

day7data['bag_parents'] = bag_parents
day7data['bag_children'] = bag_children

# create a column that will become the key - split at "contain"
print(day7data)

                                         luggage_rules     bag_parents  \
0           wavy turquoise bags contain no other bags.  wavy turquoise   
1    vibrant beige bags contain 4 drab lime bags, 1...   vibrant beige   
2    plaid green bags contain 2 pale olive bags, 1 ...     plaid green   
3    plaid fuchsia bags contain 5 dull teal bags, 4...   plaid fuchsia   
4        vibrant coral bags contain 1 dotted blue bag.   vibrant coral   
..                                                 ...             ...   
589  plaid teal bags contain 4 shiny teal bags, 2 w...      plaid teal   
590  dull violet bags contain 5 light coral bags, 1...     dull violet   
591  faded crimson bags contain 3 dark chartreuse b...   faded crimson   
592  posh bronze bags contain 4 plaid lavender bags...     posh bronze   
593  clear olive bags contain 2 muted gray bags, 2 ...     clear olive   

                                          bag_children  
0                                           [no other]

In [130]:
# Now I have my keys in one column, and a list of values in another.
# I'm going to...sigh...loop through each value and create a key value pair based on each.


#lugggage_dictionary = ''

#for x in range(len(day7data)):
#    print(len(day7data.iloc[x]['luggage_values']))
#    for z in range(len(day7data.iloc[x]['luggage_values'])):
#        print('hi')


# First, look for the color as the value

luggage_color = ['shiny gold']
bag_count = 0
backtrack_list = []

for x in range(len(day7data)):
    for z in range(len(luggage_color)):
        if luggage_color[z] in day7data.iloc[x]['bag_children']:
            bag_count += 1
            backtrack_list.append(day7data.iloc[x]['bag_parents'])

print(bag_count)
print(backtrack_list)
#    print(len(day7data.iloc[x]['luggage_values']))
#    for z in range(len(day7data.iloc[x]['luggage_values'])):
#        print('hi')


13
['plaid purple', 'posh tomato', 'striped fuchsia', 'posh tan', 'plaid gray', 'dotted beige', 'clear tomato', 'mirrored yellow', 'bright olive', 'drab gold', 'posh salmon', 'shiny plum', 'posh bronze']


In [131]:
# Option 2 - break apart the lists of children into individual rows

pairwise_df = pd.DataFrame()
bag_parent_list = []
bag_child_list = []

for x in range(len(day7data)):
    for z in range(len(day7data.iloc[x]['bag_children'])):
        bag_parent_list.append(day7data.iloc[x]['bag_parents'])
        bag_child_list.append(day7data.iloc[x]['bag_children'][z])

pairwise_df['bag_parent'] = bag_parent_list
pairwise_df['bag_child'] = bag_child_list

print(len(pairwise_df))
print(pairwise_df)


1500
          bag_parent      bag_child
0     wavy turquoise       no other
1      vibrant beige      drab lime
2      vibrant beige   muted violet
3      vibrant beige      drab plum
4      vibrant beige   shiny silver
...              ...            ...
1495     posh bronze   shiny indigo
1496     clear olive     muted gray
1497     clear olive       dark red
1498     clear olive    clear brown
1499     clear olive  bright silver

[1500 rows x 2 columns]


In [133]:
def number_of_bags(bag_color,master_list):
    # parents will keep a list of all the parents of the bag color we're looking up so that we can rerun the fuction
    # master_list is the overall list of all bag colors that could contain the original bag color
    parents = []
    for z in range(len(bag_color)):
        # filter pairwise_df down to any rows that have the bag color as the child
        subset = pairwise_df[pairwise_df.bag_child == bag_color[z]]
        # filter pairwise_df down to any rows that have the bag color as the child
        parents.extend(subset['bag_parent'].tolist())
        parents = list(set(parents))
        master_list.extend(parents)
    # If there are parents we need to look up, run the function again and pass in master_list as the second argument
    if len(parents) > 0:
        number_of_bags(parents,master_list)
    return(list(set(master_list)))

final_master_list = number_of_bags(['shiny gold'],[])
print(len(final_master_list))

222


In [ ]:
#def number_of_bags(bag_color):
#    # parents will keep a list of all the parents of the bag color we're looking up so that we can rerun the fuction
#    parents = []
#    # master_list is the overall list of all bag colors that could contain the original bag color
#    master_list = []
#    for z in range(len(bag_color)):
#        print('z: ',z)
#        # filter pairwise_df down to any rows that have the bag color as the child
#        subset = pairwise_df[pairwise_df.bag_child == bag_color[z]]
#        # filter pairwise_df down to any rows that have the bag color as the child
#        parents.extend(subset['bag_parent'].tolist())
#        parents = list(set(parents))
#        master_list.extend(parents)
#        master_list = list(set(master_list))
#        print(parents)
#        print(master_list)
#
#number_of_bags(['shiny gold'])

In [ ]:
day4data['passport_no'] = 0

passport_number = 1

for x in range(len(day4data)):
    if (pd.isnull(day4data.iloc[x]['passport_data'])):
    # If value is null, increment the passport number
        passport_number +=1
        day4data['passport_no'].iloc[x] = 0
    else:
    # If value is not null, apply the current group number
        day4data['passport_no'].iloc[x] = passport_number

manifesto = pd.DataFrame()

# For manifesto, start two blanks lists that we'll append to, then they can become dataframe columns
passport_number = []
# Passport detail will be the concatenated text for each individual passport
passport_detail = []

# Setting the range this way to get rid of the zeroes
for x in range(1,day4data['passport_no'].max()+1):
    passport_number.append(x)
    subset = day4data[day4data.passport_no == x]
    passport_string = ''
    for y in range(len(subset)):
        passport_string = passport_string + ' ' + str(subset.iloc[y]['passport_data'])
    passport_detail.append(passport_string.strip())
        
# Now append those lists as columns
manifesto['passport_number'] = passport_number
manifesto['passport_detail'] = passport_detail

# End result is a list of all passport numbers, with all of the fields in one lin

In [127]:
# For the first challenge, go through that passport_detail field and see if each field name is found

# Entries is a counter for how many of the fields are found for each passport
entries = []
# For the second part, I'm going to create a dictionary that should make it easy to extract the value associated with each key
passport_dict = []
search_list = ['byr:','iyr:','eyr:','hgt:','hcl:','ecl:','pid:']


for z in range(len(manifesto)):
    # build a dictionary by splitting at spaces, then separating the key from value using the colon
    d = dict(x.split(":") for x in manifesto.iloc[z]['passport_detail'].split(" "))
    entry = 0
    # count the number of passport fields for the passport
    for search_item in search_list:
        if search_item in manifesto.iloc[z]['passport_detail']:
            entry +=1
    entries.append(entry)
    passport_dict.append(d)
    
manifesto['passport_dict'] = passport_dict
manifesto['entries'] = entries
del manifesto['passport_detail']
        
# Create a total number of True values by row for every column in [search_list]
#manifesto['passport_fields'] = manifesto[search_list].sum(axis=1)    

print(manifesto.head())

# The number of rows with the same valid fields as the length of the search_list is the correct answer
print('part 1 answer:',len(manifesto[manifesto.entries == len(search_list)]))

   passport_number                                      passport_dict  entries
0                1  {'iyr': '2015', 'hgt': '59cm', 'byr': '2029', ...        7
1                2  {'ecl': 'blu', 'iyr': '2018', 'pid': '94361475...        5
2                3  {'pid': '067285985', 'hcl': '#ceb3a1', 'cid': ...        7
3                4  {'hgt': '185cm', 'ecl': 'gry', 'cid': '222', '...        7
4                5  {'pid': '260043570', 'hcl': '#b6652a', 'cid': ...        6
part 1 answer: 210


In [129]:
# For the second challenge, set up individual functions to call and test each of the requirements

#byr (Birth Year) - four digits; at least 1920 and at most 2002.
def byr_test(byr_value):
    if int(byr_value) >= 1920 and int(byr_value) <= 2002:
        valid = 1
    else:
        valid = 0
    return(valid)

#iyr (Issue Year) - four digits; at least 2010 and at most 2020.
def iyr_test(iyr_value):
    if int(iyr_value) >= 2010 and int(iyr_value) <= 2020:
        valid = 1
    else:
        valid = 0
    return(valid)

#eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
def eyr_test(eyr_value):
    if int(eyr_value) >= 2020 and int(eyr_value) <= 2030:
        valid = 1
    else:
        valid = 0
    return(valid)

#ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
def ecl_test(ecl_value):
    ecl_list = ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']
    if ecl_value in ecl_list:
        valid = 1
    else:
        valid = 0
    return(valid)

#hgt (Height) - a number followed by either cm or in:
#If cm, the number must be at least 150 and at most 193.
#If in, the number must be at least 59 and at most 76.
def hgt_test(hgt_value):
    try:
        system = hgt_value[-2:]
        measurement = int(hgt_value[:-2])
        if system == 'cm':
            if measurement >= 150 and measurement <= 193:
                valid = 1
            else:
                valid = 0
        elif system == 'in':
            if measurement >= 59 and measurement <= 76:
                valid = 1
            else:
                valid = 0
        else:
            valid = 0
    except:
        valid = 0
    return(valid)

#hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
def hcl_test(hcl_value):
    character_list = '0123456789abcdef'
    first_char = hcl_value[:1]
    rest_of_chars = hcl_value[1:]
    if first_char == '#' and len(rest_of_chars) == 6 and all(item in list(character_list) for item in list(rest_of_chars)) is True:
        valid = 1
    else:
        valid = 0
    return(valid)

#pid (Passport ID) - a nine-digit number, including leading zeroes.
def pid_test(pid_value):
    if len(pid_value) == 9 and pid_value.isdecimal() is True:
        valid = 1
    else:
        valid = 0
    return(valid)

search_list = ['byr:','iyr:','eyr:','hgt:','hcl:','ecl:','pid:']

# The creation of the dictionary removed colons, so that also needs to be done to loop through the keys
new_search_list = [s.replace(":", "") for s in search_list]

# valid_entries keeps a counter of how many passport properties are valid according to the conditions in their functions
valid_entries = []
# key_counter keeps a list of how many times each property passes, so we can check if a certain property is always failing
key_counter = []
key_pass_counter = []

for z in range(len(manifesto)):
    valid_counter = 0
    for key in new_search_list:
        if key in manifesto.iloc[z]['passport_dict']:
            key_counter.append(key)
            function_name = key + '_test'
            function_parameter = manifesto.iloc[z]['passport_dict'][key]
            # if the key exists, eval will run the key-specific function with value for its parameter
            valid = eval(function_name+'("'+function_parameter+'")')
            if valid == 1:
                key_pass_counter.append(key)
            valid_counter += valid
    valid_entries.append(valid_counter)

# Now just append that list of valid entry counts to the manifesto data frame
manifesto['valid_entries'] = valid_entries

# # Answer is the number of records where all keys in the search list have valid entries
print('part 2 answer:',len(manifesto[manifesto.valid_entries == len(search_list)]))

key_counter=collections.Counter(key_counter)
key_pass_counter=collections.Counter(key_pass_counter)

part 2 answer: 131


In [124]:
# And for a final bonus available only to my premium subscribers, let's get a success rate for the individual passport fields

# Turn the counters into regular dictionaries
key_counter = dict(key_counter)
key_pass_counter = dict(key_pass_counter)
key_pass_rate = {}

# Create a third dictionary of the % success rate
for key in key_pass_counter:
    key_pass_rate[key] = round(key_pass_counter[key]/key_counter[key]*100,2)
print(key_pass_rate)

{'iyr': 81.01, 'hcl': 76.23, 'byr': 79.55, 'eyr': 78.65, 'ecl': 75.84, 'pid': 76.74, 'hgt': 76.95}
